# Cantera classes for multicomponent and reacting flows

In this jupyter notebook, we are going to compute some quantities of interest using the thermochemical library Cantera. Using classes, we are going to create objects for Multicomponent and reacting flows.

In [1]:
import numpy as np
#!pip install cantera
import matplotlib.pyplot as plt
import cantera as ct
#import csv
import pandas as pd
from csv import reader  # for reading csv files
from csv import writer  # for writing csv files

First we create a class for multicomponent flow, where we set the fuel, oxidizer, equivalence ratio, and chemical mechanism as a attribute of an object.<br>
Then, we can set the thermodynamic state using the function SetTDState at a given pressure and temperature, this return a gas with the previous properties at that specific pressure and temperature. The we can create some method in order to retrieve chemical properties such as density, heat capacity, viscosity,and others.

In [2]:
class MulticomponentCantera:
    """
    Class for defining a Cantera object for multicomponent flows.

    """
    """
    Constructor attributes:
    - param phi: equivalence ratio
      type phi: float
    - param X_fuel: Dictionary that contains chemical species and its mole fraction of the fuel.
      type X_fuel: dict()
    - param X_oxidizer: Dictionary that contains chemical composition and mole fraction of the oxidizer.
      type X_oxidize: dict()
    - param chemical_mechanism: chemical mechanism in .yaml format 
      type chemical_mechanism: str
    - param phase_name: name of the phase in the .yaml file
      type phase_name: str
    - param transport_model: transport model for thermodynamics properties
      type transport_model: str
    """
    phi=0.7
    X_fuel={"H2":1.0}
    X_oxidizer={"O2":1.0, "N2":3.76}
    chemical_mechanism="h2o2.yaml"
    phase_name="ohmech"
    transport_model="mixture-averaged"

    def __init__(self, phi, X_fuel, X_oxidizer, chemical_mechanism, phase_name, transport_model):
        self.phi=phi
        self.X_fuel=X_fuel
        self.X_oxidizer=X_oxidizer
        self.chemical_mechanism=chemical_mechanism
        self.phase_name=phase_name
        self.transport_model=transport_model
    """
    Set Thermodynamic state
    - param temperature: Temperature of the gas
      type temperature: float
    - param pressure_thermodynamics: operating pressure of the gas
      type pressure_thermodynamics: float
    - output gas: gas with the specified temperature and pressure.
      type gas: Cantera object
    """
    def SetTDState(self, temperature, pressure_thermodynamics):
        gas = ct.Solution(self.chemical_mechanism, self.phase_name, self.transport_model)
        gas.set_equivalence_ratio(self.phi,self.X_fuel,self.X_oxidizer)
        gas.TP=temperature, pressure_thermodynamics

        return gas
    """
    Methods for retrieving Thermochemical properties of the gas at a given composition, temperature and thermodynamic pressure
    - param gas: gas object at specific temperature and pressure
      type: Cantera object
    - method GetDensity: retrieve density of the gas mixture at a given temperature and pressure.
      return float density
    - method HeatCapacity: retrieve specific heat capacity at a constant pressure of the gas mixture at a given temperature 
      and composition.
      return float HeatCapacity
    - method GetViscosity: retrieve laminar viscosity of the gas mixture at a given temperature and composition.
      return float viscosity
    - method GetThermalConductivity: retrieve thermal conductivity of the gas mixture at a given temperature and composition.
      return float viscosity
    - method GetMassFractionComposition: retrieve species mass fractions of the gas mixture.
      return list(float)
    - method GetChemicalComposition: retrieve chemical composition of the gas mixture.
      return list(str)
    - method GetEnthalpy: retrieve total enthalpy (sensible+chemical enthalpies) of the gas mixture at a given temperature 
      and composition.
      return float Enthalpy
    """
    def GetDensity(self,gas):
        return gas.density
    def GetHeatCapacity(self,gas):
        return gas.cp_mass
    def GetViscosity(self,gas):
        return gas.viscosity
    def GetThermalConductivity(self,gas):
        return gas.thermal_conductivity
    def GetMassFractionComposition(self,gas):
        return list(gas.mass_fraction_dict().values())
    def GetChemicalComposition(self,gas):
        return list(gas.mass_fraction_dict().keys())
    def GetEnthalpy(self, gas):
        return gas.enthalpy_mass


Instantiate a MulticomponentCantera Object for testing and retrieving its thermochemical properties

In [4]:
Test = MulticomponentCantera(0.7, {"H2":1.0}, {"O2":1.0, "N2":3.76}, "h2o2.yaml", "ohmech", "mixture-averaged")
gas=Test.SetTDState(300,101325.0)
print(Test.GetDensity(gas))
print(Test.GetHeatCapacity(gas))
print(Test.GetViscosity(gas))
print(Test.GetMassFractionComposition(gas))
print(Test.GetChemicalComposition(gas))
print(Test.GetEnthalpy(gas))

0.9242362170821292
1277.9100866538756
1.8454886433731362e-05
[0.020137986304114414, 0.7515544436281938, 0.22830757006769173]
['H2', 'N2', 'O2']
2402.1845860029007


# Class for multicomponent report used for writing .csv file with thermochemical properties at given temperature and pressure

Class Report inherits attributes and methods from MulticomponentCantera class

In [5]:
class Report(MulticomponentCantera):
    """
    Class Report inherits the attributes and method from the Parent class MulticomponentCantera
    Constructor attributes:
    - param ratios: equivalence ratios used to obtain final report
      type ratios: list(float)
    - param temperature: Temperature of the gas composition at which report will be written.
      type temperature: float
    - param pressure_thermodynamics: Operating pressure of the gas composition at which report will be written.
      type X_oxidize: dict()
    """
    def __init__(self, phi, X_fuel, X_oxidizer, chemical_mechanism, phase_name, transport_model, temperature, pressure_thermodynamics, ratios):
        super().__init__(phi, X_fuel, X_oxidizer, chemical_mechanism, phase_name, transport_model)
        self.ratios=ratios
        self.temperature=temperature
        self.pressure_thermodynamics=pressure_thermodynamics
    @classmethod
    def from_multicomponentCantera(cls, multicomponentCantera):
        return cls(multicomponentCantera.phi, multicomponentCantera.X_fuel, multicomponentCantera.X_oxidizer,multicomponentCantera.chemical_mechanism,multicomponentCantera.phase_name,multicomponentCantera.tranmsport_model)
    """
    method for writing report
    - param name: name of the .csv file, if no option is given, default name "H2" is added to the file
      type name: str
    """
    def reportfile(self, name=None):
        default_name='H2'
        name = default_name if name is None else name
        with open('Cantera_Properties_'+name+'.csv', 'w', newline='\n') as write_obj:
            for i in range(0,len(self.ratios)):
                properties =[]
                self.phi=self.ratios[i]
                gas=MulticomponentCantera.SetTDState(self,self.temperature, self.pressure_thermodynamics)
                density=MulticomponentCantera.GetDensity(self, gas)
                viscosity=MulticomponentCantera.GetViscosity(self, gas)
                thermal_conductivity=MulticomponentCantera.GetThermalConductivity(self,gas)
                Cp=MulticomponentCantera.GetHeatCapacity(self, gas)
                massFractions=MulticomponentCantera.GetMassFractionComposition(self,gas)
                total_enthalpy=MulticomponentCantera.GetEnthalpy(self,gas)
                if(i==0):
                    properties_name=["phi"]
                    properties_name.extend(MulticomponentCantera.GetChemicalComposition(self,gas))
                    properties_name.extend(["density","Cp","viscosity","thermal_conductivity", "total enthalpy"])
                    writer(write_obj).writerow(properties_name)
                properties.append(self.phi)
                properties.extend(massFractions)
                properties.extend([density,Cp,viscosity,thermal_conductivity, total_enthalpy])
                writer(write_obj).writerow(properties)

First example: use Report class using default name

In [6]:
# defining ratios at which you want to retrieve thermochemical properties
phi_ratios=[0.3,0.4,0.5,0.6,0.7,0.8]
# define temperature for the gas mixture
temperature=300.0
# define operating pressure
pressure=101325.0
# instantiate class report
test_report=Report(Test.phi, Test.X_fuel, Test.X_oxidizer,Test.chemical_mechanism,
                   Test.phase_name, Test.transport_model, temperature,pressure,phi_ratios)

In [7]:
# create report file with default name H2
test_report.reportfile()

Second example: for different fuel and oxidizer, write a new report

In [8]:
Test.X_fuel = {"CH4":1.0}
Test.X_oxidizer={"O2": 2.0, "N2": 7.52}
Test.chemical_mechanism="1S_methane.yaml"
Test.phase_name="CH4_MP1_mix"
Test.transport_model="mixture-averaged"

In [9]:
test_report=Report(Test.phi, Test.X_fuel, Test.X_oxidizer,Test.chemical_mechanism,Test.phase_name, Test.transport_model, temperature,pressure,phi_ratios)
test_report.reportfile('CH4')

# Class for Reacting flows which inherits MulticomponentCantera class and adds chemical properties

In [10]:
class ReactingCantera(MulticomponentCantera):
    @classmethod
    def from_multicomponentCantera(cls, MulticomponentCantera):
        return cls(MulticomponentCantera.phi, MulticomponentCantera.X_fuel, MulticomponentCantera.X_oxidizer,MulticomponentCantera.chemical_mechanism,
                   MulticomponentCantera.phase_name, MulticomponentCantera.transport_model)
    """
    Retrieve gas object from SetTDState method defined in MulticomponentCantera
    """
    def reaction(self,temperature, pressure):
        gas=MulticomponentCantera.SetTDState(self,temperature,pressure)
        
        return gas
    """
    method for retrieveng NetProductionRates, mass fractions and molecular weights from Cantera Library
    - param temperature: temperature of the gas mixture
      type name: float
    - param pressure: Operating pressure
      type name: float
    - output lists: a list that contains reaction rates, species mass fractions and molecular weights
      type name: list(list(float),list(float),list(float))
    """
    def getNetProductionRates(self,temperature,pressure):
        gas=ReactingCantera.reaction(self,temperature,pressure)
        reactionRates=[]
        massFractions=[]
        molecularWeights=[]
        for i in range(0,gas.n_total_species):
            reactionRates.append([gas.species_name(i),gas.net_production_rates[i]*gas.molecular_weights[i]])
            massFractions.append([gas.species_name(i),gas.mass_fraction_dict().get(gas.species_name(i))])
            molecularWeights.append([gas.species_name(i),gas.molecular_weights[i]])
        
        return reactionRates, massFractions, molecularWeights

example: instantiate ReactingCantera object at a given temperature and pressure

In [11]:
reaction_test=ReactingCantera(Test.phi, Test.X_fuel, Test.X_oxidizer,Test.chemical_mechanism, Test.phase_name, Test.transport_model)
reactionRates=reaction_test.getNetProductionRates(1700,101325)

Small check that sum of chemical reaction rates of all chemical species sum up 0 (mass conservation) <br>
$$\sum_{i=1}^{n}\dot{\omega_{i}}=0$$

In [12]:
conservation_check=0
for i in range(0,len(reactionRates[0])):
    conservation_check=+reactionRates[0][i][1]
print("chemical conservation:",conservation_check)

chemical conservation: 0.0


Use ReactingCantera class to write a .csv file to summarize chemical reacting properties of interest

In [13]:
def reportfileReactions(temperature,reactionRates):
    with open('Cantera_Reaction_Rates.csv', 'a', newline='\n') as write_obj:
        species=["temperature"]
        reactions_Rates=[temperature]
        for i in range(0,len(reactionRates)):
            species.append(reactionRates[i][0])
            reactions_Rates.append(reactionRates[i][1])
        writer(write_obj).writerow(species)
        writer(write_obj).writerow(reactions_Rates)

Example how to use reportfileReaction function

In [14]:
temperature=[700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000]
for i in range(0,len(temperature)):
    reportfileReactions(temperature[i],reaction_test.getNetProductionRates(temperature[i],101325)[0])   